# Initial Setup

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('default')

from utils.matchers import FeatureMatcher, MultipleInstanceMatcher
from utils.visualization import display_frames, display_ar_frames
from utils.utils import save_ar_video_f2r, save_ar_video_f2f, save_ar_video

#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2

# Overview of the data

In [2]:
video_filename = './Data/Multiple View.avi'

reference_frame = cv2.cvtColor(cv2.imread('Data/ReferenceFrame.png', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
reference_mask = cv2.imread('Data/ObjectMask.PNG', cv2.IMREAD_GRAYSCALE)

ar_layer = cv2.cvtColor(cv2.imread('Data/AugmentedLayer.PNG', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)[:,:640]
ar_mask = cv2.imread('Data/AugmentedLayerMask.PNG', cv2.IMREAD_GRAYSCALE)[:,:640]


## Reference frame

In [ ]:
w, h, dpi = 1000, 500, 100
fig, ax = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(reference_frame)
ax[0].set_title('Reference frame')

ax[1].imshow(reference_mask, cmap='gray')
ax[1].set_title('Mask')

fig.tight_layout(pad=0.5)
plt.show()

## AR layer

In [ ]:
w, h, dpi = 1000, 500, 100
fig, ax = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(ar_layer)
ax[0].set_title('AR layer')

ax[1].imshow(ar_mask, cmap='gray')
ax[1].set_title('Mask')

fig.tight_layout(pad=0.5)
plt.show()

## Video sequence

In [ ]:
display_frames(video_filename, starting_frame=0)

# Stitch AR layer onto reference frame

In [ ]:
ar_frame = reference_frame.copy()
ar_frame[ar_mask==255] = ar_layer[ar_mask==255]

w, h, dpi = 1200, 400, 100
fig, ax = plt.subplots(ncols=3, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(reference_frame)
ax[0].set_title('Reference frame')

ax[1].imshow(ar_layer)
ax[1].set_title('AR layer')

ax[2].imshow(ar_frame)
ax[2].set_title('Combination')

fig.tight_layout(pad=0.5)
plt.show()

# Do AR on the video
Overlay the AR layer onto the video frame.

At the beginning, the AR layer and a reference image onto which the AR layer should be stitched are needed.

Then the homography between the reference image and the video frame is computed using local invariant features, and it is then applied to the AR layer. The feature algorithms tested here are:
 - SIFT: accurate, but slow
 - BRISK: faster, but less accurate
 - ORB: even faster, even less accurate

The homography between the reference image and the video frames is found using local invariant features. The matches between the descriptors are then using FLANN KDTree. The homography on the resulting matches is computed using RANSAC.

All this is implemented in the ``matchers.FeatureMatcher`` class.

To find the correct homography between the reference image and the video frame 3 approaches are tried:
 - F2R (frame to reference): here, the homography between the reference image and each video frame is computed. Here, it is harder to find the right correspondance between the reference image and the video frame. The result is jittering of the AR layer, which is made worse by using a low accuracy feature matching algorithm. As can be seen in the result video, SIFT produces the best results here, albeit with a performance hit.
 - F2F (frame to frame): the homography between the reference image and the first video frame is computed first (in this specific example, it is the identity), and then the homographies between each pair of subsequent frames is computed. This tends to produce smoother results, but the AR layer drifts over time. Less accurate algorithms produce more drift
 - Hybrid: every 30 frames, a F2F correspondance using SIFT is found, then a F2R with a faster algorithm (BRISK in this case) is done. This has the benefit of both approaches, as it enables to have the performance of the faster algorithm, with limited drift.

Having said this, visually the best results are still achieved with F2R using SIFT.

In [3]:
video_filename = './Data/Multiple View.avi'

reference_frame = cv2.cvtColor(cv2.imread('Data/ReferenceFrame.png', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
reference_mask = cv2.imread('Data/ObjectMask.PNG', cv2.IMREAD_GRAYSCALE)

ar_layer = cv2.cvtColor(cv2.imread('Data/AugmentedLayer.PNG', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)[:,:640]
ar_mask = cv2.imread('Data/AugmentedLayerMask.PNG', cv2.IMREAD_GRAYSCALE)[:,:640]


## F2R

SIFT feature matching

In [ ]:
save_ar_video_f2r(video_filename,
                  'out_f2r_sift.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask,
                  algorithm_f2r=cv2.SIFT_create())

BRISK feature matching

In [ ]:
save_ar_video_f2r(video_filename,
                  'out_f2r_brisk.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask,
                  algorithm_f2r=cv2.BRISK_create())

ORB feature matching

In [ ]:
save_ar_video_f2r(video_filename,
                  'out_f2r_orb.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask,
                  algorithm_f2r=cv2.ORB_create())

## F2F

SIFT feature matching

In [ ]:
save_ar_video_f2f(video_filename,
                  'out_f2f_sift.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask,
                  algorithm_f2f=cv2.SIFT_create())

BRISK feature matching

In [ ]:
save_ar_video_f2f(video_filename,
                  'out_f2f_brisk.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask,
                  algorithm_f2f=cv2.BRISK_create())

ORB feature matching

In [ ]:
save_ar_video_f2f(video_filename,
                  'out_f2f_orb.avi',
                  ar_layer=ar_layer,
                  ar_mask=ar_mask,
                  reference_image=reference_frame,
                  reference_mask=reference_mask,
                  algorithm_f2f=cv2.ORB_create())

## Hybrid
F2F using BRISK with F2R every 30 frames using SIFT

In [ ]:
r = 30
save_ar_video(video_filename,
              f'out_r{r}.avi',
              ar_layer=ar_layer,
              ar_mask=ar_mask,
              reference_image=reference_frame,
              reference_mask=reference_mask,
              drift_correction_step=r,
              algorithm_f2r=cv2.SIFT_create(),
              algorithm_f2f=cv2.BRISK_create())